In [9]:
import subprocess
import csv
import os
import sys
import re
import time

def run_test(window_size, error_rate, filename):
    os.makedirs('logs', exist_ok=True)
    log_file = 'logs/performance_log.csv'
    
    # Prepare log file if not exists
    if not os.path.exists(log_file):
        with open(log_file, 'w', newline='') as csvfile:
            csv.writer(csvfile).writerow(['error_rate', 'window_size', 'retransmissions', 'total_time'])
    
    # Use separate terminal windows for server and client
    server_cmd = [
        sys.executable, 'server.py'
    ]
    client_cmd = [
        sys.executable, 'client.py'
    ]
    
    # Open log file for writing server output
    with open('server_output.txt', 'w') as server_log:
        # Start server process
        server_process = subprocess.Popen(
            server_cmd, 
            stdin=subprocess.PIPE, 
            stdout=server_log, 
            stderr=subprocess.PIPE, 
            text=True
        )
        
        # Provide server inputs
        server_process.stdin.write(f"{window_size}\n{filename}\n{error_rate}\n")
        server_process.stdin.flush()
        
        # Small delay to ensure server is ready
        time.sleep(1)
        
        # Start client process
        client_process = subprocess.Popen(
            client_cmd, 
            stdin=subprocess.PIPE, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE, 
            text=True
        )
        
        # Provide client inputs
        client_process.stdin.write(f"{filename}\n{error_rate}\n")
        client_process.stdin.flush()
        
        # Wait for processes to complete
        server_process.wait(timeout=10)
        client_process.wait(timeout=10)
    
    # Read server output
    with open('server_output.txt', 'r') as server_log:
        server_output = server_log.read()
    
    # Extract metrics
    retransmissions_match = re.search(r'Total retransmissions:\s*(\d+)', server_output)
    time_match = re.search(r'Total time of the process:\s*([\d.]+)', server_output)
    
    retransmissions = retransmissions_match.group(1) if retransmissions_match else 'N/A'
    total_time = time_match.group(1) if time_match else 'N/A'
    
    # Log results
    with open(log_file, 'a', newline='') as csvfile:
        csv.writer(csvfile).writerow([
            error_rate, 
            window_size, 
            retransmissions, 
            total_time
        ])
    
    print(f"Test complete: Error Rate {error_rate}, Window Size {window_size}")

def main():
    window_sizes = [1, 2, 4, 8]
    error_rates = [10, 30, 50]
    filename = 'zero'
    
    for window_size in window_sizes:
        for error_rate in error_rates:
            run_test(window_size, error_rate, filename)

if __name__ == "__main__":
    main()

Test complete: Error Rate 10, Window Size 1
Test complete: Error Rate 30, Window Size 1
Test complete: Error Rate 50, Window Size 1
Test complete: Error Rate 10, Window Size 2
Test complete: Error Rate 30, Window Size 2
Test complete: Error Rate 50, Window Size 2
Test complete: Error Rate 10, Window Size 4
Test complete: Error Rate 30, Window Size 4
Test complete: Error Rate 50, Window Size 4
Test complete: Error Rate 10, Window Size 8
Test complete: Error Rate 30, Window Size 8
Test complete: Error Rate 50, Window Size 8


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the performance log
df = pd.read_csv('server_log.csv')

# Set style for better visualization
sns.set_style('whitegrid')

# 1. Error Rate vs Total Time
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='error_rate', y='total_time', hue='window_size', palette='viridis')
plt.title('Error Rate vs Total Transmission Time')
plt.xlabel('Error Rate (%)')
plt.ylabel('Total Transmission Time (s)')
plt.savefig('logs/error_rate_vs_time.png')
plt.close()

# 2. Window Size vs Total Time
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='window_size', y='total_time', hue='window_size', palette='rocket')
plt.title('Window Size vs Total Transmission Time')
plt.xlabel('Window Size')
plt.ylabel('Total Transmission Time (s)')
plt.savefig('logs/window_size_vs_time.png')
plt.close()

# 3. Error Rate vs Retransmissions
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='error_rate', y='retransmissions', hue='window_size', palette='coolwarm')
plt.title('Error Rate vs Number of Retransmissions')
plt.xlabel('Error Rate (%)')
plt.ylabel('Number of Retransmissions')
plt.savefig('logs/error_rate_vs_retransmissions.png')
plt.close()

print("Visualizations generated in logs directory.")

Visualizations generated in logs directory.
